#### Objetive of the Task reduce the Trainable param 8,000 & accuracy is above 99.4%"

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import np_utils

In [2]:
mnist=tf.keras.datasets.mnist

In [3]:
(x_train_full,y_train_full),(x_test,y_test)=mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
(x_train,x_valid)=x_train_full[5000:],x_train_full[:5000]
(y_train,y_valid)=y_train_full[5000:],y_train_full[:5000]

In [5]:
x_train.shape

(55000, 28, 28)

In [6]:
num_classes=len(np.unique(y_train))


y_train=np_utils.to_categorical(y_train,num_classes)

y_test=np_utils.to_categorical(y_test,num_classes)
y_valid=np_utils.to_categorical(y_valid,num_classes)

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Activation

from keras.layers import BatchNormalization,Dropout, LeakyReLU
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

In [8]:
# CONV-BATCH-ACTIVATION_POOLING
model=Sequential()

model.add(Conv2D(filters=32,kernel_size=3,input_shape=(28,28,1))) #input - Kernal+1 =>26x26x8
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(MaxPooling2D(pool_size=2)) #12x12x16
model.add(Conv2D(filters=14,kernel_size=3)) #input - Kernal+1 =>26x26x8
model.add(BatchNormalization())
model.add(Activation('LeakyReLU'))
model.add(MaxPooling2D(pool_size=2)) #5x5x16

model.add(Conv2D(filters=10,kernel_size=5)) #5x5x10
model.add(Flatten())

model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 32)       128       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 26, 26, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 14)        4046      
                                                                 
 batch_normalization_1 (Batc  (None, 11, 11, 14)       5

In [9]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    min_delta=0.005,
    baseline=0.98,
    patience=10,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=5,
    factor=0.2,
    min_lr=1e-6,
    verbose=1
)

In [10]:
callbacks = [
    #early_stopping,
    lr_scheduler,
]

In [17]:
#Here weight /Kernal will update
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'],
    )

In [18]:
#from keras.callbacks import ModelCheckpoint,EarlyStopping,LearningRateScheduler
EPOCHS=50
VALIDATION_DATA=(x_valid,y_valid)

model.fit(x_train,y_train,epochs=EPOCHS,validation_data=VALIDATION_DATA,callbacks=callbacks,
    use_multiprocessing=True)

Epoch 1/50
1719/1719 [==============================] - 63s 36ms/step - loss: 0.1908 - accuracy: 0.9450 - val_loss: 0.0619 - val_accuracy: 0.9820 - lr: 0.0010
Epoch 2/50
1719/1719 [==============================] - 63s 37ms/step - loss: 0.0631 - accuracy: 0.9810 - val_loss: 0.0522 - val_accuracy: 0.9840 - lr: 0.0010
Epoch 3/50
1719/1719 [==============================] - 63s 36ms/step - loss: 0.0504 - accuracy: 0.9843 - val_loss: 0.0697 - val_accuracy: 0.9784 - lr: 0.0010
Epoch 4/50
1719/1719 [==============================] - 64s 37ms/step - loss: 0.0428 - accuracy: 0.9867 - val_loss: 0.0429 - val_accuracy: 0.9888 - lr: 0.0010
Epoch 5/50
1719/1719 [==============================] - 63s 37ms/step - loss: 0.0376 - accuracy: 0.9880 - val_loss: 0.0539 - val_accuracy: 0.9844 - lr: 0.0010
Epoch 6/50
1719/1719 [==============================] - 64s 37ms/step - loss: 0.0343 - accuracy: 0.9887 - val_loss: 0.0550 - val_accuracy: 0.9846 - lr: 0.0010
Epoch 7/50
1719/1719 [========================

In [ ]:
# evaluate test accuracy
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)